In [1]:
from src.main.flags_global import FLAGS
FLAGS.mark_as_parsed()
from src.main.RecognizeElevatorElements import RecognizeElevatorElements
from absl import logging

Error: Does not exist sound mixer library.
   > pip3 install pygame
Error: Does not exist ffmpeg.
   > brew install ffmpeg # on MacOSX
   or
   > sudo apt install -y ffmpeg # on Ubuntu


In [2]:
label_map = FLAGS.label_map_path_detection
application = RecognizeElevatorElements()

In [37]:
def get_next_input_batch():
    input_data = application.input.get_input_data_batch()
    if input_data is None:
        logging.warning('No InputData. Skipping.')
        return
    return input_data

def next_image_detection(input_image):
    return application.detection.detect_next_image(input_image)

def nms_detection(detections):
    return application.processing.filter_by_nms(detections)

def filter_only_floor_buttons(detections_nms):
    return application.processing.filter_one_category(detections_nms, category='btn_floor')

from src.utils.image_utils import crop_multiple_images_by_bndbox
def crop_by_bndbox(input_image, detections_to_crop):
    return crop_multiple_images_by_bndbox(input_image, detections_to_crop)

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def classify_buttons(detection_buttons_roi):
    button_classifications = []
    for button_roi in detection_buttons_roi:
        button_roi_tensor = tf.convert_to_tensor(np.expand_dims(button_roi, 0), dtype=tf.float32)
        button_roi_tensor_resized = tf.image.resize_with_pad(button_roi_tensor, 224, 224, method=tf.image.ResizeMethod.BICUBIC)
        button_classification = application.classification.classify_next_tensor(button_roi_tensor_resized)
        button_classifications.append(button_classification)
    return button_classifications

def sort_button_scores_and_assign_labels(button_classifications, labels):
    predictions_sorted = []
    for entry in button_classifications:
        prediction = entry[0]
        scores_sorted, labels_sorted = application.processing.get_classification_label(prediction, labels)
        predictions_sorted.append({'class_prediction': labels_sorted, 'class_score': scores_sorted})
    return predictions_sorted

In [40]:
input_data = get_next_input_batch()

In [41]:
detections = next_image_detection(input_data['ImageData'])
detections_nms = nms_detection(detections)
detection_buttons = filter_only_floor_buttons(detections_nms)

In [42]:
detection_buttons_roi = crop_by_bndbox(input_data['ImageData'], detection_buttons['detection_boxes_nms'])
button_classifications = classify_buttons(detection_buttons_roi)

In [39]:
labels = [-2, -1, 0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 3, 4, 5, 6, 7, 8, 9]
button_classifications_sorted = sort_button_scores_and_assign_labels(button_classifications, labels)

In [ ]:
#TODO: visualize the outputs :D